# [``anydyce``](https://posita.github.io/anydyce/latest/) Visualization Examples

Selected examples highlighting ``dyce``’s use and capabilities. Select ☞ ``Cell`` ☞ ``Run All`` from the menu above to run them.

## AnyDice replacement

``anydyce`` affords functionality similar to [AnyDice](https://anydice.com/) within [Jupyter](https://jupyter.org/) via its ``anydyce.jupyter_visualize`` interface.

In [ ]:
import warnings
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    try:
        import anydyce
    except (ImportError, ModuleNotFoundError):
        import sys
        !{sys.executable} -m pip install --upgrade pip
        !{sys.executable} -m pip install anydyce ipywidgets matplotlib
        import anydyce
print("Ready!")

### Interactive example: highest, middle, and lowest of 3d6

In [ ]:
from anydyce import BreakoutType, jupyter_visualize
from dyce import P

p_3d6 = 3@P(6)
jupyter_visualize(
    [
        p_3d6.h(-1),  # highest
        p_3d6.h(1),  # middle
        p_3d6.h(0),  # lowest
    ],
    default_breakout_type=BreakoutType.BARH,
    default_markers="o",
)

### Interactive example: Powered by the _Apocalypse_ (PbtA)

Expected distributions from _[Apocalypse World](http://apocalypse-world.com/)’s_ core mechanic with various modifiers.

In [ ]:
from enum import IntEnum, auto
from anydyce import BreakoutType, jupyter_visualize
from dyce import H

class PBTA(IntEnum):
    FAIL = 0
    COST = auto()
    SUCC = auto()

def pbta(h, outcome):
    if outcome <= 6:
        return PBTA.FAIL
    elif outcome >= 10:
        return PBTA.SUCC
    else:
        return PBTA.COST

jupyter_visualize(
    [
        (f"PBTA{mod:+d}", ((2@H(6)) + mod).substitute(pbta), ((2@H(6)) + mod))
        for mod in range(-1, 4)
    ], 
    default_breakout_type=BreakoutType.BURST,
    default_main_plot_type="bar",
)

## Lower level visualization conveniences

For more detailed control over output, ``anydyce`` exposes additional visualization conveniences.

### Visualization example: 2d10 vs. d8+d12, bar, line, and burst charts

Comparing two mechanics with similar distributions via ``anydyce``’s take on a bar chart (``anydyce.viz.plot_bar``), a line chart (``anydyce.viz.plot_line``), and a “burst” chart (``anydyce.viz.display_burst`` which is ``anydyce``’s take on a donut chart).

In [ ]:
from anydyce.viz import display_burst, plot_bar, plot_line
from dyce import H
import matplotlib.pyplot
import matplotlib.style

matplotlib.style.use("bmh")
matplotlib.pyplot.rcParams["figure.figsize"] = (12, 12)

d10_2 = 2@H(10)
d10_2_label = "2d10"
d8d12 = H(8) + H(12)
d8d12_label = "d8+d12"

# Burst chart
ax = matplotlib.pyplot.subplot2grid((2, 2), (0, 0), colspan=2)
display_burst(
    ax,
    h_inner=d10_2,
    h_outer=d8d12,
    inner_color="Purples_r",
    outer_color="Greens_r",
    alpha=0.8,
    title=f"Outer: {d8d12_label}\nInner: {d10_2_label}",
)

# Bar chart
ax = matplotlib.pyplot.subplot2grid((2, 2), (1, 0))
plot_bar(ax, [
    (d10_2_label, d10_2),
    (d8d12_label, d8d12),
], alpha=0.8)
# Re-color the bars
so_far = 0
for count, color in zip(
    (len(h) for h in (d10_2, d8d12)),
    ("tab:purple", "tab:green"),
):
    for i in range(count):
        ax.patches[i + so_far].set_color(color)
    so_far += count
ax.legend()

# Line chart
ax = matplotlib.pyplot.subplot2grid((2, 2), (1, 1))
plot_line(ax, [
    (d10_2_label, d10_2),
    (d8d12_label, d8d12),
], markers="<>", alpha=0.8)
# Re-color the lines
ax.lines[0].set_color("tab:purple")
ax.lines[1].set_color("tab:green")
ax.legend()

matplotlib.pyplot.show()